In [ ]:
import json
import os
from dotenv import load_dotenv
import subprocess as sp
import pandas as pd
from collections import defaultdict
from datetime import datetime

In [ ]:
load_dotenv()

True

In [ ]:
github_token = os.getenv("GITHUB_ACCESS_TOKEN")
sonarqube_token = os.getenv("SONARQUBE_ACCESS_TOKEN_LESS")
sonarqube_server_url = os.getenv("SONARQUBE_SERVER_URL")

In [ ]:
demo_github_slug = "less"
demo_repo_name = "less"
demo_repo_path = "C:/SWE Class/Github Desktop/TestProjects/less.js"

In [ ]:
clean_code_attribute_categories = [
    "RESPONSIBLE",
    "ADAPTABLE",
    "CONSISTENT",
    "INTENTIONAL",
]

severities = ["LOW", "MEDIUM", "HIGH"]

quality_attributes = [
    "RELIABILITY",
    "SECURITY",
    "MAINTAINABILITY",
]

code_metrics = [
    "sqale_debt_ratio",
    "vulnerabilities",
    "security_rating",
    "sqale_rating",
    "complexity",
    "cognitive_complexity",
    "ncloc",
]

indexing_fields = ["version", "release_date", "timestamp"]

code_metrics_query_string = ",".join(code_metrics)
# code_metrics_query_string

In [ ]:
metrics_dict = defaultdict(list)

for metric in indexing_fields + code_metrics:
    metrics_dict[metric] = []

metrics_dict

defaultdict(list,
            {'version': [],
             'release_date': [],
             'timestamp': [],
             'sqale_debt_ratio': [],
             'vulnerabilities': [],
             'security_rating': [],
             'sqale_rating': [],
             'complexity': [],
             'cognitive_complexity': [],
             'ncloc': []})

In [ ]:
clean_code_attribute_dict = defaultdict()

for category in clean_code_attribute_categories:
    clean_code_attribute_dict[category] = defaultdict(list)

clean_code_attribute_dict

defaultdict(None,
            {'RESPONSIBLE': defaultdict(list, {}),
             'ADAPTABLE': defaultdict(list, {}),
             'CONSISTENT': defaultdict(list, {}),
             'INTENTIONAL': defaultdict(list, {})})

In [ ]:
sonarqube_token, sonarqube_server_url

('sqp_6b348f3e01cb80f409e2ce39994afe6cb8a1f5d6', 'http://localhost:9000')

## Git CLI Data Extraction


In [ ]:
def get_tag_list(repo_path: str, max_output: int = 20) -> list[str]:
    # Subprocess to get the git directory
    sp.run(
        ["git", "rev-parse", "--git-dir"],
        cwd=repo_path,
        capture_output=True,
        check=True,
    )

    # Subprocess to get the git tags in the repo
    result = sp.run(
        ["git", "tag", "-l"],
        cwd=repo_path,
        capture_output=True,
        check=True,
    )

    tag_list = result.stdout.decode("utf-8").split("\n")
    tag_list = [tag for tag in tag_list if tag != ""]

    if max_output is not None and len(tag_list) > max_output:
        tag_list = tag_list[-max_output:]

    return tag_list

In [ ]:
def get_tag_and_timestamp(repo_path: str) -> list[tuple[str, str, int]]:
    tag_list = get_tag_list(repo_path, None)
    tag_and_timestamp = []

    for tag in tag_list:
        result = sp.run(
            ["git", "log", "-1", "--format=%cd %ct", "--date=short", tag],
            cwd=repo_path,
            capture_output=True,
            check=True,
            text=True,
        )

        date, timestamp = result.stdout.strip().split()
        tag_and_timestamp.append((tag, date, int(timestamp)))
        # print(result.stdout.strip())
    return tag_and_timestamp

In [ ]:
def get_version_tags_by_year(data: list[tuple[str, str, int]], max_per_year=5):
    versions_by_year = defaultdict(list)
    datetime_converted_data = [
        (version, datetime.strptime(date, "%Y-%m-%d"), timestamp)
        for version, date, timestamp in data
    ]

    for version, release_date, timestamp in datetime_converted_data:
        year = release_date.year
        versions_by_year[year].append((version, release_date, timestamp))

    selected_versions = []

    for year in sorted(versions_by_year.keys()):
        versions_of_this_year = versions_by_year[year]
        selected_versions.extend(
            versions_of_this_year[: min(max_per_year, len(versions_of_this_year))]
        )

    return [
        (version, release_date.strftime("%Y-%m-%d"), timestamp)
        for version, release_date, timestamp in selected_versions
    ]

In [ ]:
tags_and_timestamps = get_version_tags_by_year(get_tag_and_timestamp(demo_repo_path))
len(tags_and_timestamps)

26

In [ ]:
# Checkout to a specific tag and show the output of the operation
def checkout_to_tag(repo_path: str, tag: str):
    result = sp.run(
        ["git", "checkout", tag],
        cwd=repo_path,
        capture_output=True,
        check=True,
        text=True,
    )
    print(result.stdout.strip())

In [ ]:
# Checkout to a specific tag use sonar-scanner to scan the project
def sonar_scan(repo_path: str, tag: str):
    result = sp.run(
        [
            "sonar-scanner",
            "-D",
            f"sonar.projectBaseDir={demo_repo_path}",
            "-D",
            "sonar.projectVersion=" + tag,
            "-D",
            f"sonar.projectKey={demo_repo_name}",
            "-D",
            "sonar.sources=.",
            "-D",
            f"sonar.host.url={sonarqube_server_url}",
            "-D",
            "sonar.login=" + sonarqube_token,
        ],
        cwd=repo_path,
        capture_output=True,
        check=True,
        text=True,
        shell=True,
    )
    # print(result.stdout)

In [ ]:
# Call the sonarqube api to get the metrics of the project
def get_metrics() -> dict:
    result = sp.run(
        [
            "curl.exe",
            f"{sonarqube_server_url}/api/measures/component?component={demo_repo_name}&metricKeys={code_metrics_query_string}",
        ],
        capture_output=True,
        check=True,
        text=True,
        # shell=True,
    )
    return json.loads(result.stdout)

In [ ]:
# Get metrics of the project
def get_metrics_from_version_tags(
    tag: str, date: str, timestamp: int, output_dict=metrics_dict
):
    metrics_result = get_metrics()

    output_dict["version"].append(tag)
    output_dict["release_date"].append(date)
    output_dict["timestamp"].append(timestamp)

    for metric in metrics_result["component"]["measures"]:
        output_dict[metric["metric"]].append(metric["value"])

    return output_dict

In [ ]:
# A mock call to test sonar-scanner
sp.run(
    ["sonar-scanner", "-h"],
    capture_output=True,
    check=True,
    text=True,
    shell=True,
)

CompletedProcess(args=['sonar-scanner', '-h'], returncode=0, stdout='INFO: \nINFO: usage: sonar-scanner [options]\nINFO: \nINFO: Options:\nINFO:  -D,--define <arg>     Define property\nINFO:  -h,--help             Display help information\nINFO:  -v,--version          Display version information\nINFO:  -X,--debug            Produce execution debug output\n', stderr='')

## Issues Narrowing from SonarQube


In [ ]:
# A function that can convert given time string into minutes. The time string may include hours, minutes and seconds. It is in the following format: "1d2h3min". The days, hours or minutes may be missing. If the string is empty, return 0.


def time_str_to_minutes(time: str) -> int:
    if time == "":
        return 0

    time = time.replace("d", " ").replace("h", " ").replace("min", " ").split()
    time = [int(t) for t in time]

    if len(time) == 1:
        return time[0]
    elif len(time) == 2:
        return time[0] * 60 + time[1]
    elif len(time) == 3:
        return time[0] * 24 * 60 + time[1] * 60 + time[2]
    else:
        raise ValueError("Invalid time string")

In [ ]:
def get_quality_issue_values_from_api(clean_code_category: str, severity: str):
    """
    return a tuple of (total_security_issues, total_reliability_issues, total_maintainability_issues, total_debt)
    """

    result = sp.run(
        [
            "curl.exe",
            f"{sonarqube_server_url}/api/issues/search?cleanCodeAttributeCategories={clean_code_category}",
        ],
        capture_output=True,
        check=True,
        text=True,
        # shell=True,
    )

    issues_list = json.loads(result.stdout)["issues"]

    # Get the software quality, maintainability and debt from the issues list
    filtered_issues_list = [
        {
            "software_quality_category": issue["impacts"][0]["softwareQuality"],
            "severity": issue["impacts"][0]["severity"],
            "debt": time_str_to_minutes(issue["debt"]),
        }
        for issue in issues_list
        if issue["type"] == "CODE_SMELL" and issue["impacts"][0]["severity"] == severity
    ]

    # Get the total number of security, reliability and maintainability quality issue count and debt
    total_security_issues = len(
        [
            issue
            for issue in filtered_issues_list
            if issue["software_quality_category"] == "SECURITY"
        ]
    )
    total_reliability_issues = len(
        [
            issue
            for issue in filtered_issues_list
            if issue["software_quality_category"] == "RELIABILITY"
        ]
    )
    total_maintainability_issues = len(
        [
            issue
            for issue in filtered_issues_list
            if issue["software_quality_category"] == "MAINTAINABILITY"
        ]
    )

    total_debt = sum([issue["debt"] for issue in filtered_issues_list])

    return (
        total_security_issues,
        total_reliability_issues,
        total_maintainability_issues,
        total_debt,
    )

In [ ]:
def extract_clean_code_data(tag: str, date: str, timestamp: int):
    for clean_code_category in clean_code_attribute_categories:
        clean_code_attribute_dict[clean_code_category]["version"].append(tag)
        clean_code_attribute_dict[clean_code_category]["date"].append(date)
        clean_code_attribute_dict[clean_code_category]["timestamp"].append(timestamp)

        for severity in severities:
            (
                total_security_issues,
                total_reliability_issues,
                total_maintainability_issues,
                total_debt,
            ) = get_quality_issue_values_from_api(clean_code_category, severity)

            clean_code_attribute_dict[clean_code_category][
                f"security_issues_{severity.lower()}"
            ].append(total_security_issues)

            clean_code_attribute_dict[clean_code_category][
                f"reliability_issues_{severity.lower()}"
            ].append(total_reliability_issues)

            clean_code_attribute_dict[clean_code_category][
                f"maintainability_issues_{severity.lower()}"
            ].append(total_maintainability_issues)

            clean_code_attribute_dict[clean_code_category][
                f"total_debt_{severity.lower()}"
            ].append(total_debt)

In [ ]:
for tag_and_timestamp in tags_and_timestamps:
    tag, date, _ = tag_and_timestamp
    checkout_to_tag(demo_repo_path, tag)
    sonar_scan(demo_repo_path, tag)
    get_metrics_from_version_tags(*tag_and_timestamp)
    extract_clean_code_data(*tag_and_timestamp)

In [ ]:
metrics_dataset = pd.DataFrame(metrics_dict, index=None)
metrics_dataset

,version,release_date,timestamp,sqale_debt_ratio,vulnerabilities,security_rating,sqale_rating,complexity,cognitive_complexity,ncloc
0,v0.4.0,2013-02-18,1361206207,1.6,0,1.0,1.0,712,407,3753
1,v0.4.1,2013-03-13,1363184208,1.6,0,1.0,1.0,772,413,3939
2,v0.4.2,2013-12-16,1387222558,1.6,0,1.0,1.0,799,447,4025
3,_pr1108_author_head,2014-03-31,1396233124,1.6,0,1.0,1.0,725,408,3819
4,_pr1170_author_head,2014-06-20,1403285855,1.6,0,1.0,1.0,602,362,3541
5,v0.4.3,2014-03-07,1394216677,1.6,0,1.0,1.0,724,408,3815
6,v0.4.4,2014-03-12,1394656115,1.6,0,1.0,1.0,724,408,3815
7,v0.4.5,2014-05-12,1399916630,1.6,0,1.0,1.0,591,355,3479
8,v1.0.0,2016-04-04,1459811577,1.6,0,1.0,1.0,676,421,3734
9,v1.0.0-rc1,2016-02-11,1455212775,1.6,0,1.0,1.0,609,369,3582


In [ ]:
metrics_dataset.to_csv(f"../Datasets/RQ1/{demo_repo_name}_metrics.csv")

In [ ]:
for clean_code_attribute, attribute_dict in clean_code_attribute_dict.items():
    dataframe = pd.DataFrame(attribute_dict, index=None)
    dataframe.to_csv(
        f"../Datasets/RQ2/{demo_repo_name}/{demo_repo_name}_{clean_code_attribute.lower()}_issues.csv",
        index=False,
    )